In [1]:
import numpy as np
from dadk.QUBOSolverCPU import *
from dadk.BinPol import *
from datetime import datetime
import pandas as pd

# Input Data

In [3]:
# Input data 

# Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].values.tolist()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].values.tolist()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].values.tolist()

n_assets = len(asset_quantity)

# Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].values.tolist()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].values.tolist()

n_accounts = len(account_exposure)

# Haircuts
haircuts = (pd.read_csv("data/sample_haircuts.csv", header=None).values.tolist())

#Single Limits
#single_limits = (pd.read_csv("data/sample_single_limits.csv", header=None).values.tolist())

# Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

 ## Penalty Terms


In [3]:
# Penalty terms
lambda_cost_fn = 1e2
lambda_consistency1, lambda_consistency2 = 1e11, 1e9
lambda_exposure1, lambda_exposure2 = 1, 1

#lambda_single_lim1, lambda_single_lim2 = 1,1

# Constructing the QUBO

In [4]:
# n-bit binary variables
n_binary = 7
binary_coeff = []
max_binary_value = (2**n_binary) - 1

for b in range(1,n_binary+1):
    aux_term = (2**(b-1) * 1) / max_binary_value
    binary_coeff.append(aux_term)

binary_variables = VarShapeSet(BitArrayShape(name='bin_vars', shape=(n_assets, n_accounts, n_binary), axis_names=['Assets', 'Accounts', 'Binary']))


In [5]:
# Constructing QUBO equation

# Objective function
cost_function = BinPol(binary_variables)
for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            cost_function.add_term(cost_factor_matrix[i][j]*binary_coeff[k],(("bin_vars", i, j, k),))

In [6]:
#consistency requirement
con1 = BinPol(binary_variables)
con2 = BinPol(binary_variables)

con_constraint = BinPol(binary_variables)
for i in range(n_assets):
    aux_term = BinPol(binary_variables)
    for j in range(n_accounts):
        for k in range(n_binary):
            aux_term.add_term(binary_coeff[k],(("bin_vars", i, j, k),))
        aux_term.add_term(-1,())
    con1.add(aux_term)
    aux_term.power(2)
    con2.add(aux_term)

con1.multiply_scalar(lambda_consistency1)
con2.multiply_scalar(lambda_consistency2)

con_constraint = con1 + con2


In [7]:
# Exposure requirements
exposure1  = BinPol(binary_variables)
exposure2 = BinPol(binary_variables)


for j in range(n_accounts):
    aux_term = BinPol(binary_variables)
    for i in range(n_assets):
        for k in range(n_binary):
            aux_term.add_term(binary_coeff[k]*asset_quantity[i]*asset_value[i]*haircuts[i][j]/100,(("bin_vars", i, j, k),))
    aux_term.add_term(-account_exposure[j],())
    exposure1.add(aux_term)
    aux_term.power(2)
    exposure2.add(aux_term)

exposure1.multiply_scalar(lambda_exposure1)
exposure2.multiply_scalar(lambda_exposure2)

exposure_requirements = -exposure1 + exposure2


In [ ]:
# #single_limits
# single1 = BinPol(var_shape_set)
# single2  = BinPol(var_shape_set)


# for i in range(n_assets):
#     for j in range(n_accounts):
#         step = BinPol(var_shape_set)
#         for k in range(nbit):
#             step.add_term(binary_coeff[k]*asset_quantity[i],(("bin_vars", i, j, k),))
#         step.add_term(-single_limits[i][j],())
#         single1.add(step)
#     step.power(2)
# #         print(step)
#     single2.add(step)
# #     print(step)
# #     print('--')
        
# single1.multiply_scalar(lambda_single_lim1)
# single2.multiply_scalar(lambda_single_lim2)

# single_limit = single1 + single2


In [8]:
# Final QUBO
final_equation = (cost_function*lambda_cost_fn + con_constraint + exposure_requirements) #+ single_limit

#To include single limits into the qubo uncomment all lines refering to them and the additional validation below

# Run Solver

In [9]:
solver = QUBOSolverCPU(
   
    number_iterations    = 1000,               # total number of itrations per run
    number_runs          = 1000,                 # number of stochastically independant runs
    temperature_start    = 5000,               # start temperature for annealing as float value
    temperature_end      = 10,                 # end temperature for annealing as float value 
    temperature_mode     = 0,                         # 0: reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                                      # 1: reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                                      # 2: reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    temperature_decay    = 0.0095,             # see temperature_mode 0
    temperature_interval = 1,                  # see temperature_mode 0
    offset_increase_rate = 5000.0,             # increase of dynamic offset when no bit selected, set to 0.0 to switch off dynamic offset
    graphics             = True                # create data for graphics output
)


In [10]:
# Run solver
start=datetime.now()
solution_list = solver.minimize(final_equation)

# Get results
solution = solution_list.get_minimum_energy_solution()
configuration = solution.configuration
my_bit_array = solution.extract_bit_array("bin_vars")

# Verify Results

In [11]:
bitstring = ''

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=(2**k)*my_bit_array[i][j][k]
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]/100
    allocation_solution[i] = asset


=====Each Individual Allocation Percentage=====
Assign 0.00% of asset 1 to account 1
Assign 0.00% of asset 1 to account 2
Assign 0.00% of asset 1 to account 3
Assign 0.00% of asset 1 to account 4
Assign 0.00% of asset 1 to account 5
Assign 0.00% of asset 2 to account 1
Assign 0.00% of asset 2 to account 2
Assign 0.00% of asset 2 to account 3
Assign 0.00% of asset 2 to account 4
Assign 0.79% of asset 2 to account 5
Assign 0.00% of asset 3 to account 1
Assign 0.00% of asset 3 to account 2
Assign 0.00% of asset 3 to account 3
Assign 0.00% of asset 3 to account 4
Assign 0.00% of asset 3 to account 5
Assign 0.00% of asset 4 to account 1
Assign 0.00% of asset 4 to account 2
Assign 0.00% of asset 4 to account 3
Assign 0.00% of asset 4 to account 4
Assign 0.00% of asset 4 to account 5
Assign 7.87% of asset 5 to account 1
Assign 14.17% of asset 5 to account 2
Assign 3.15% of asset 5 to account 3
Assign 0.79% of asset 5 to account 4
Assign 13.39% of asset 5 to account 5
Assign 25.98% of asset 6 

In [12]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

====Validation for Consistency====
0.00% of asset 1 posted
0.79% of asset 2 posted
0.00% of asset 3 posted
0.00% of asset 4 posted
39.37% of asset 5 posted
62.20% of asset 6 posted
0.00% of asset 7 posted
12.60% of asset 8 posted
0.00% of asset 9 posted
0.00% of asset 10 posted


In [13]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i],:} required")


=====Validation for Exposure======
$797,017.4540605295 of collateral posted, $824000.0 required
$658,710.4124126025 of collateral posted, $669292.0 required
$525,925.7566234448 of collateral posted, $548600.0 required
$36,391.48292141409 of collateral posted, $64844.52 required
$444,886.22703210777 of collateral posted, $472000.0 required


In [ ]:
# #print("===Validation of Single limits =====")
# for i in range(n_assets):
#     for j in range(n_accounts):
#         decimal = 0
#         for k in range(n_binary):
#             decimal +=(2**k)*my_bit_array[i][j][k]
#         allocation = (decimal/max_binary_value)*asset_quantity[i] 
#         print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
#         print(f"Constraint Satisified?: {allocation <= single_limits[i][j]}")

In [17]:
print(f"The objective function is: {cost_function.compute(configuration)}")
print("The optimal objective function is: 0.4299816571359823")

The objective function is: 0.5393700787401574
The optimal objective function is: 0.4299816571359823


In [15]:
print(datetime.now()-start)

0:10:47.681934


# Export to .csv

In [16]:
# Export to .csv

Q_value_balanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))


for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=binary_coeff[k]*my_bit_array[i][j][k]*max_binary_value
        asset += decimal*(100/max_binary_value)
        Q_value_balanced_py_qubo[i][j] = decimal/max_binary_value

#Uncomment the line below to save the Data to csv

# pd.DataFrame(Q_value_balanced_py_qubo).to_csv("Q_value_unbalanced_fujitsu_no_single_lim3.csv", header=None, index=None)